In [1]:
import os
import tensorflow as tf
tf.enable_eager_execution()
import pandas as pd
import math
import numpy as np
import itertools
import collections

In [2]:
from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset.utils import  frame_utils
from waymo_open_dataset import dataset_pb2 as open_dataset

In [3]:
# get all segment file names and save in Filenames
def scan(folder):        
    x=[]
    for path,dirs,files in os.walk(folder):
        for f in files:
            if f.startswith('segment'):
                x.append(os.path.join(path,f))
    return(x)

folder = './waymo-od/data'

Filenames=scan(folder)

In [4]:
def getID(f):         #get the ID of segment from file name
    f = f.replace('_with_camera_labels.tfrecord', '')
    f = f.split('segment-',1)[1]
    return f

In [176]:
### Extract segment files in Frames, each file consists of multiple frames

Frames = {}

for file in Filenames[1:2]:
    SegID = getID(file)
    Segment = []
    dataset = tf.data.TFRecordDataset(file, compression_type='')
    for data in dataset:
        frame = open_dataset.Frame()
        frame.ParseFromString(bytearray(data.numpy()))
        Segment.append(frame)
    Frames[SegID] = Segment

## Each frame includes:
#### 1. ***context***: similar for all the frames in one segment
#### 2. ***timestamp_micros*** : should be devided to 1000000
#### 3. ***pose***
#### 4. ***laser_labels*** : includes
#### 5. ***camera_labels***

In [177]:
def CountLLabels(frame):    #number of laser labels in a frame
    return len(frame.laser_labels)

def converttime(t):
    t = t/1000000
    t = datetime.fromtimestamp(t)
    return t

In [178]:
def Ped2df(SegID, tstamp, pedLoc, 
           pedMeta,VehPos,tday,w):
    
    length = len(pedLoc)
    t, px, py, pz, pw, pl, pht, phd, psx, psy, paccx, paccy, vpose = (
        [] for i in range(13)
    )
    
    for i in range(length):
        t.append(tstamp[i])
        px.append(pedLoc[i].center_x)
        py.append(pedLoc[i].center_y)
        pz.append(pedLoc[i].center_z)
        pw.append(pedLoc[i].width)       
        pl.append(pedLoc[i].length)
        pht.append(pedLoc[i].height)
        phd.append(pedLoc[i].heading)
        psx.append(pedMeta[i].speed_x)
        psy.append(pedMeta[i].speed_y)
        paccx.append(pedMeta[i].accel_x)
        paccy.append(pedMeta[i].accel_y)
        vpose.append(VehPos[i].transform)
        
    data = np.array([t, px, py, pz, pw, pl, pht, phd,
                     psx, psy, paccx, paccy, vpose]).T.tolist()
    columnsnames= ['Timestamp', 'x', 'y', 
              'z', 'Width', 'Length',
              'Height','Heading', 'Speed x',
              'Speed y', 'accel x', 'accel y', 'vpose']
    pedDF = pd.DataFrame(data, columns = columnsnames)
    
    transform = pedDF['vpose'].apply(pd.Series)
    transform = transform.rename(columns= lambda x: 'AVtransform_' +str(x))
    pedDF = pedDF.drop(columns= 'vpose')
    pedDF = pd.concat([pedDF[:],transform[:]],axis=1)
    
    pedDF['Weather'] = w
    pedDF['Time of Day'] = tday
    pedDF['Segment ID'] = SegID
    return pedDF
 

In [181]:
def PedExtract(SegID):
    pedLoc = collections.defaultdict(list)                             #pedestrians label box
    pedMeta = collections.defaultdict(list)                            #pedestrians label metadata
    VehPos =  collections.defaultdict(list)                            #ego veh pose
    tstamp = collections.defaultdict(list) 
    tday = Frames[SegID][0].context.stats.time_of_day
    w = Frames[SegID][0].context.stats.weather
    
    for j in range(len(Frames[SegID])):
        t = Frames[SegID][j].timestamp_micros                      #timestamp
        vpos = Frames[SegID][j].pose                                #Vehicle position
        Nlabels = CountLLabels(Frames[SegID][j])                   # number of laser labels
       
        for k in range(Nlabels):
            LidarLab = Frames[SegID][j].laser_labels[k]
            if  LidarLab.type == 2:                      # type 2: pedestrian
                
                PedID = LidarLab.id

                tstamp[PedID].append(t)
                pedLoc[PedID].append(LidarLab.box)

                
                pedMeta[PedID].append(LidarLab.metadata)
                
                
                VehPos[PedID].append(Frames[SegID][j].pose)
    
    Ped = {}
    for p in list(pedLoc.keys()):
        Ped[p] = Ped2df(SegID, tstamp[p], pedLoc[p],
                        pedMeta[p],VehPos[p],tday, w)
        
    
    return Ped      

In [ ]:
for i in range(len(Frames)):
    SegID = list(Frames.keys())[i]
    PedExtract(SegID)
    

In [81]:
px, py, pz, pl, pht, phd, psx, psy, paccx, paccy, vpose = ([] for i in range(11))

In [ ]:
Frames[SegID][0].context.time_of_day
Frames[SegID][0].context.weather


In [ ]:
list(Frames[SegID][0].context.stats.laser_object_counts)

In [ ]:
np.reshape(np.array(Frames[SegID][0].pose.transform), [4, 4])